# Selecting a multiphase model in Ansys Fluent

**NOTE:** this notebook is simply a support to model selection based on a few notes taken from Ansys Fluent Theory Guide 2022R1. Do not base your model selection simply on these assumptions, other factors and domain knowledge being the most relevant factors.

In [1]:
import numpy as np


def material_density_ratio(rho_d, rho_c):
    """ Ansys Fluent Theory Guide 2022R1 (14.2). """
    return rho_d / rho_c


def particulate_loading_two_phase(alpha_d, gamma):
    """ Ansys Fluent Theory Guide 2022R1 (14.1). """
    return gamma * alpha_d / (1 - alpha_d)


def particle_distance_crowe(alpha_d):
    """ Ansys Fluent Theory Guide 2022R1 (14.3).

    See Crowe et al [116] Chapter 2 for details.
    """
    return pow(np.pi / (6 * alpha_d), 1/3)


def particle_response_time(rho_d, diam, mu_c):
    """ Response time for a particle in a fluid. """
    return rho_d * pow(diam, 2) / (18 * mu_c)


def system_response_time(L_s, V_s):
    """ Response time for a a fluid. """
    return L_s / V_s


def stokes_number(rho_d, diam, mu_c, L_s, V_s):
    """ Particle to system response time ratio (Stokes number). """
    tau_d = particle_response_time(rho_d, diam, mu_c)
    tau_c = system_response_time(L_s, V_s)

    print(f"\n**** TIME RESPONSES ****")
    print(f"Particle response time ............. {tau_d:.6f} s")
    print(f"System response time ............... {tau_c:.6f} s")

    return tau_d / tau_c


def multiphase_model(rho_c: float, rho_d: float, alpha_d: float,
                     diam: float, mu_c: float, L_s: float, V_s: float,
                     combustion: bool = False, granular: bool = False,
                     stratified_or_slug: bool = False,
                     beta_lims: list[float] = [0.001, 0.1],
                     stokes_lims: list[float] = [0.01, 0.9, 1.1]):
    """ Helper to select a multiphase model/approach. """
    gamma = material_density_ratio(rho_d, rho_c)
    beta = particulate_loading_two_phase(alpha_d, gamma)
    dist = particle_distance_crowe(alpha_d)
    st = stokes_number(rho_d, diam, mu_c, L_s, V_s)

    print(f"\n**** BASE REPORT ****")
    print(f"Material density ratio ............. {gamma:.6f}")
    print(f"Particulate loading factor ......... {beta:.6f}")
    print(f"Characteristic particle distance ... {dist:.6f}")
    print(f"Stokes number ...................... {st:.6f}")

    print(f"\n**** SUGGESTED APPROACH ****")

    if st < stokes_lims[0]:
        print(f"No conclusion based on low Stokes number {st:.6f}")
        print(f"Disregarding other factors, probably use mixture model")
    elif st >= stokes_lims[1] and st < stokes_lims[2]:
        print(f"No conclusion based on Stockes number {st:.6f} ~ 1")
    else:
        print(f"Particle flow is independent of fluid with Stokes {st:.6f}")
        print(f"Adapted models in this case are DPM or Eulerian model")

    if combustion:
        print(f"Only DPM model is adapted to particulate combustion")
        return
    
    if granular:
        print(f"Disregading other factors, probably use Eulerian model")

    if stratified_or_slug:
        print(f"Stratified/slug flow must be treated with VOF")
        return

    if beta <= beta_lims[0]:
        print(f"1-way coupling: low loading detected = {beta:.6f}")
        print(f"Recommended models are DPM or mixture model")
    elif beta <= beta_lims[1]:
        print(f"2-way coupling: intermediate loading detected = {beta:.6f}")
        print(f"Possible models are DPM or Eulerian model")
    else:
        print(f"4-way coupling: high loading detected = {beta:.6f}")
        print(f"Only Eulerian model is compatible")

## Example - coal classifier

In [2]:
rho_d = 1448.0
rho_c = 1.2
alpha_d = 0.001
diam = 30e-06
mu_c = 1.81e-05
L_s = 1.0
V_s = 10.0

multiphase_model(rho_c, rho_d, alpha_d, diam, mu_c, L_s, V_s,
                 combustion=False, granular=True,
                 stratified_or_slug=False)


**** TIME RESPONSES ****
Particle response time ............. 0.004000 s
System response time ............... 0.100000 s

**** BASE REPORT ****
Material density ratio ............. 1206.666667
Particulate loading factor ......... 1.207875
Characteristic particle distance ... 8.059960
Stokes number ...................... 0.040000

**** SUGGESTED APPROACH ****
Particle flow is independent of fluid with Stokes 0.040000
Adapted models in this case are DPM or Eulerian model
Disregading other factors, probably use Eulerian model
4-way coupling: high loading detected = 1.207875
Only Eulerian model is compatible


## Example - mineral processing

In [3]:
rho_d = 3200.0
rho_c = 1.2
alpha_d = 0.5
diam = 300e-06
mu_c = 0.032
L_s = 0.2
V_s = 2.0

multiphase_model(rho_c, rho_d, alpha_d, diam, mu_c, L_s, V_s,
                 combustion=False, granular=True,
                 stratified_or_slug=False)


**** TIME RESPONSES ****
Particle response time ............. 0.000500 s
System response time ............... 0.100000 s

**** BASE REPORT ****
Material density ratio ............. 2666.666667
Particulate loading factor ......... 2666.666667
Characteristic particle distance ... 1.015491
Stokes number ...................... 0.005000

**** SUGGESTED APPROACH ****
No conclusion based on low Stokes number 0.005000
Disregarding other factors, probably use mixture model
Disregading other factors, probably use Eulerian model
4-way coupling: high loading detected = 2666.666667
Only Eulerian model is compatible


## Example - coal combustion

In [4]:
rho_d = 1448.0
rho_c = 0.3
alpha_d = 0.0001
diam = 30e-06
mu_c = 4.4e-05
L_s = 1.0
V_s = 10.0

multiphase_model(rho_c, rho_d, alpha_d, diam, mu_c, L_s, V_s,
                 combustion=True, granular=True,
                 stratified_or_slug=False)


**** TIME RESPONSES ****
Particle response time ............. 0.001645 s
System response time ............... 0.100000 s

**** BASE REPORT ****
Material density ratio ............. 4826.666667
Particulate loading factor ......... 0.482715
Characteristic particle distance ... 17.364657
Stokes number ...................... 0.016455

**** SUGGESTED APPROACH ****
Particle flow is independent of fluid with Stokes 0.016455
Adapted models in this case are DPM or Eulerian model
Only DPM model is adapted to particulate combustion


## Example - water spray

In [5]:
rho_d = 1000.0
rho_c = 1.2
alpha_d = 0.0001
diam = 10e-06
mu_c = 1.81e-05
L_s = 1.0
V_s = 10.0

multiphase_model(rho_c, rho_d, alpha_d, diam, mu_c, L_s, V_s,
                 combustion=False, granular=False,
                 stratified_or_slug=False)


**** TIME RESPONSES ****
Particle response time ............. 0.000307 s
System response time ............... 0.100000 s

**** BASE REPORT ****
Material density ratio ............. 833.333333
Particulate loading factor ......... 0.083342
Characteristic particle distance ... 17.364657
Stokes number ...................... 0.003069

**** SUGGESTED APPROACH ****
No conclusion based on low Stokes number 0.003069
Disregarding other factors, probably use mixture model
2-way coupling: intermediate loading detected = 0.083342
Possible models are DPM or Eulerian model


## Example - water and oil

In [6]:
rho_d = 800.0
rho_c = 1000.0
alpha_d = 0.1
diam = 0.01
mu_c = 0.001
L_s = 1.0
V_s = 0.5

multiphase_model(rho_c, rho_d, alpha_d, diam, mu_c, L_s, V_s,
                 combustion=False, granular=False,
                 stratified_or_slug=True)


**** TIME RESPONSES ****
Particle response time ............. 4.444444 s
System response time ............... 2.000000 s

**** BASE REPORT ****
Material density ratio ............. 0.800000
Particulate loading factor ......... 0.088889
Characteristic particle distance ... 1.736466
Stokes number ...................... 2.222222

**** SUGGESTED APPROACH ****
Particle flow is independent of fluid with Stokes 2.222222
Adapted models in this case are DPM or Eulerian model
Stratified/slug flow must be treated with VOF
